***GENERATED CODE FOR lacrimets PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import pyspark
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationTimeSeriesForecastingMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]['featureList']
        ForecastFE = configObj["FE"]
        featuresSelectedList = [ForecastFE['features']
                                ['timecolumn'], ForecastFE['features']['tocompare']]
        transformedDF = transformationDF.select(
            [c for c in transformationDF.columns if c in featuresSelectedList])
        transformedDF = CleanseData().replaceNullValues(featureData, transformedDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AREA NAME', 'transformation_label': 'String Indexer'}], 'feature': 'AREA NAME', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '77th Street', 'max': 'Wilshire', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AREA NAME'}, {'feature_label': 'AREA NAME', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AREA NAME')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Crm Cd Desc', 'transformation_label': 'String Indexer'}], 'feature': 'Crm Cd Desc', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ARSON', 'max': 'VIOLATION OF RESTRAINING ORDER', 'missing': '0', 'distinct': '53'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Crm Cd Desc'}, {'feature_label': 'Crm Cd Desc', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Crm Cd Desc')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Mocodes', 'transformation_label': 'String Indexer'}], 'feature': 'Mocodes', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '425', 'mean': '665.4', 'stddev': '610.9', 'min': '0100 0928', 'max': '930', 'missing': '48', 'distinct': '352'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Mocodes'}, {'feature_label': 'Mocodes', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Mocodes')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Vict Sex', 'transformation_label': 'String Indexer'}], 'feature': 'Vict Sex', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '466', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'X', 'missing': '71', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Vict Sex'}, {'feature_label': 'Vict Sex', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Vict Sex')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Vict Descent', 'transformation_label': 'String Indexer'}], 'feature': 'Vict Descent', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '466', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'X', 'missing': '34', 'distinct': '13'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Vict Descent'}, {'feature_label': 'Vict Descent', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Vict Descent')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Premis Desc', 'transformation_label': 'String Indexer'}], 'feature': 'Premis Desc', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ALLEY', 'max': 'YARD (RESIDENTIAL/BUSINESS)', 'missing': '0', 'distinct': '60'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Premis Desc'}, {'feature_label': 'Premis Desc', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Premis Desc')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Weapon Desc', 'transformation_label': 'String Indexer'}], 'feature': 'Weapon Desc', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '172', 'mean': '', 'stddev': '', 'min': 'AIR PISTOL/REVOLVER/RIFLE/BB GUN', 'max': 'VERBAL THREAT', 'missing': '397', 'distinct': '25'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Weapon Desc'}, {'feature_label': 'Weapon Desc', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Weapon Desc')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Status', 'transformation_label': 'String Indexer'}], 'feature': 'Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AA', 'max': 'JO', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Status'}, {'feature_label': 'Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Status Desc', 'transformation_label': 'String Indexer'}], 'feature': 'Status Desc', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Adult Arrest', 'max': 'Juv Other', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Status Desc'}, {'feature_label': 'Status Desc', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Status Desc')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LOCATION', 'transformation_label': 'String Indexer'}], 'feature': 'LOCATION', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '100 E  3RD                          ST', 'max': 'YUCCA                        ST', 'missing': '0', 'distinct': '492'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LOCATION'}, {'feature_label': 'LOCATION', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LOCATION')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Cross Street', 'transformation_label': 'String Indexer'}], 'feature': 'Cross Street', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '79', 'mean': '', 'stddev': '', 'min': '103RD                        ST', 'max': 'ZELZAH', 'missing': '468', 'distinct': '74'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Cross Street'}, {'feature_label': 'Cross Street', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Cross Street')

        transformedDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        df = transformedDF.toPandas()
        return df


***AUTOML FUNCTIONS***

In [ ]:
from prophet import Prophet
import pandas as pd


def driverProphet(df):
    df.columns = ['ds', 'y']
    df['ds'] = df['ds'].astype(str).str[:-6]
    df['ds'] = pd.to_datetime(df['ds'])
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=365)
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    m.plot(forecast)
    return m


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run lacrimetsHooks.ipynb
try:
	#sourcePreExecutionHook()

	lacrimedatafromtopresent = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/LACrime_Data_from_2020_to_Present.csv', 'filename': 'LACrime_Data_from_2020_to_Present.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(lacrimedatafromtopresent)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run lacrimetsHooks.ipynb
try:
	#transformationPreExecutionHook()

	lacrimetsfeatureforecast = TransformationTimeSeriesForecastingMain.run(lacrimedatafromtopresent, json.dumps( {"FE": {"functionList": [{"function": "Original"}], "stage_attributes": {}, "featureList": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "DR_NO", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "201453435.58", "stddev": "1962242.69", "min": "200105738", "max": "212009108", "missing": "0"}, "updatedLabel": "DR_NO"}, {"transformationsData": [{}], "feature": "Date Rptd", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Date Rptd"}, {"transformationsData": [{}], "feature": "DATE OCC", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DATE OCC"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TIME OCC", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1145.75", "stddev": "625.54", "min": "1", "max": "2359", "missing": "0"}, "updatedLabel": "TIME OCC"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AREA", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "11.12", "stddev": "5.84", "min": "1", "max": "21", "missing": "0"}, "updatedLabel": "AREA"}, {"transformationsData": [{"feature_label": "AREA NAME", "transformation_label": "String Indexer"}], "feature": "AREA NAME", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "77th Street", "max": "Wilshire", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AREA NAME"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Rpt Dist No", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1129.7", "stddev": "590.33", "min": "111", "max": "2183", "missing": "0"}, "updatedLabel": "Rpt Dist No"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Part 1-2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.42", "stddev": "0.49", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Part 1-2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Crm Cd", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "523.97", "stddev": "211.87", "min": "110", "max": "956", "missing": "0"}, "updatedLabel": "Crm Cd"}, {"transformationsData": [{"feature_label": "Crm Cd Desc", "transformation_label": "String Indexer"}], "feature": "Crm Cd Desc", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ARSON", "max": "VIOLATION OF RESTRAINING ORDER", "missing": "0", "distinct": "53"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Crm Cd Desc"}, {"transformationsData": [{"feature_label": "Mocodes", "transformation_label": "String Indexer"}], "feature": "Mocodes", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "425", "mean": "665.4", "stddev": "610.9", "min": "0100 0928", "max": "930", "missing": "48", "distinct": "352"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Mocodes"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Vict Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "32.37", "stddev": "19.53", "min": "-2", "max": "91", "missing": "0"}, "updatedLabel": "Vict Age"}, {"transformationsData": [{"feature_label": "Vict Sex", "transformation_label": "String Indexer"}], "feature": "Vict Sex", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "466", "mean": "", "stddev": "", "min": "F", "max": "X", "missing": "71", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Vict Sex"}, {"transformationsData": [{"feature_label": "Vict Descent", "transformation_label": "String Indexer"}], "feature": "Vict Descent", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "466", "mean": "", "stddev": "", "min": "A", "max": "X", "missing": "34", "distinct": "13"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Vict Descent"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Premis Cd", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "286.27", "stddev": "209.6", "min": "101", "max": "964", "missing": "0"}, "updatedLabel": "Premis Cd"}, {"transformationsData": [{"feature_label": "Premis Desc", "transformation_label": "String Indexer"}], "feature": "Premis Desc", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ALLEY", "max": "YARD (RESIDENTIAL/BUSINESS)", "missing": "0", "distinct": "60"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Premis Desc"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Weapon Used Cd", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "103", "mean": "407.59", "stddev": "75.11", "min": "102", "max": "511", "missing": "397"}, "updatedLabel": "Weapon Used Cd"}, {"transformationsData": [{"feature_label": "Weapon Desc", "transformation_label": "String Indexer"}], "feature": "Weapon Desc", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "172", "mean": "", "stddev": "", "min": "AIR PISTOL/REVOLVER/RIFLE/BB GUN", "max": "VERBAL THREAT", "missing": "397", "distinct": "25"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Weapon Desc"}, {"transformationsData": [{"feature_label": "Status", "transformation_label": "String Indexer"}], "feature": "Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AA", "max": "JO", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Status"}, {"transformationsData": [{"feature_label": "Status Desc", "transformation_label": "String Indexer"}], "feature": "Status Desc", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Adult Arrest", "max": "Juv Other", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Status Desc"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Crm Cd 1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "483.98", "stddev": "215.23", "min": "121", "max": "956", "missing": "0"}, "updatedLabel": "Crm Cd 1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Crm Cd 2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "38", "mean": "956.45", "stddev": "123.3", "min": "341", "max": "998", "missing": "467"}, "updatedLabel": "Crm Cd 2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Crm Cd 3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500"}, "updatedLabel": "Crm Cd 3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Crm Cd 4", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500"}, "updatedLabel": "Crm Cd 4"}, {"transformationsData": [{"feature_label": "LOCATION", "transformation_label": "String Indexer"}], "feature": "LOCATION", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "100 E  3RD                          ST", "max": "YUCCA                        ST", "missing": "0", "distinct": "492"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LOCATION"}, {"transformationsData": [{"feature_label": "Cross Street", "transformation_label": "String Indexer"}], "feature": "Cross Street", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "79", "mean": "", "stddev": "", "min": "103RD                        ST", "max": "ZELZAH", "missing": "468", "distinct": "74"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Cross Street"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LAT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "33.87", "stddev": "2.64", "min": "0.0", "max": "34.3239", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LAT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LON", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-118.36", "stddev": "0.1", "min": "-118.6322", "max": "-118.1643", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LON"}], "features": {"timecolumn": "DATE OCC", "tocompare": "AREA NAME_stringindexer"}, "dataPercentage": "60", "originalfile": "/FileStore/platform/uploadedSourceFiles/LACrime_Data_from_2020_to_Present.csv"}}))

	#transformationPostExecutionHook(lacrimetsfeatureforecast)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run lacrimetsHooks.ipynb
try:
	#mlPreExecutionHook()

	model = driverProphet(lacrimetsfeatureforecast)

	#mlPostExecutionHook(model)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
try:
        future = model.make_future_dataframe(periods=30,freq='MS',include_history=False)
        predictedDataFrame = model.predict(future)
        display(model.plot(predictedDataFrame))
        display(model.plot_components(predictedDataFrame))
        display(predictedDataFrame)
except Exception as ex:
    logging.error(ex)

spark.stop()


